In [4]:
pip install flair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 22.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 104.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 8.0 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=f3c207c1a94033f94a4389e15c3214a5b064e690b932aedfde02a01e011ace2c
  Stored in directory: /root/.cache/p

In [14]:
!mkdir -p /root/nltk_data
!python3 -m nltk.downloader -d /root/nltk_data punkt punkt_tab

/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [15]:
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Télécharger les ressources NLTK nécessaires
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# Initialisation des outils
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Fonction de prétraitement du texte
def preprocess_text(text):
    # Convertir en minuscules
    text = text.lower()

    # Supprimer la ponctuation
    text = ''.join([char for char in text if char not in string.punctuation])

    # Tokenisation (séparation des mots)
    tokens = nltk.word_tokenize(text)

    # Suppression des stopwords et lemmatisation
    cleaned_tokens = [
        lemmatizer.lemmatize(word) for word in tokens if word not in stop_words
    ]

    # Rejoindre les tokens pour reformer le texte
    cleaned_text = ' '.join(cleaned_tokens)
    return cleaned_text

# Exemple d'application du prétraitement
sample_text = "This is an example of text preprocessing! Let's clean it up."
preprocessed_text = preprocess_text(sample_text)

print("Texte original :")
print(sample_text)
print("\nTexte prétraité :")
print(preprocessed_text)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Texte original :
This is an example of text preprocessing! Let's clean it up.

Texte prétraité :
example text preprocessing let clean


In [16]:
import flair
import csv
from tqdm import tqdm
import nltk
from nltk.corpus import stopwords

# Télécharger les stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Chargement du modèle Flair
flair_sentiment = flair.models.TextClassifier.load('en-sentiment')

# Initialisation des structures de données
polarities = dict()
occurences = dict()
categories = {"VP": [], "VN": [], "FP": [], "FN": []}

# Fichier d'entrée et paramètres
input_file = "/content/IMDB-Dataset.100.csv"
output_file = "/content/result_out.txt"

# Lecture et traitement des données
with open(input_file, newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile, delimiter=',', quotechar='"')
    next(reader)
    row_num = 0

    for row in reader:
        row_num += 1
        critique = row[0]
        true_label = row[1].strip().lower() 

      
        filtered_critique = preprocess_text(critique)

        s = flair.data.Sentence(filtered_critique)
        flair_sentiment.predict(s)
        predicted_label = s.labels[0].value.lower()
        sentence_score = s.labels[0].score

        # Catégorisation des critiques
        if true_label == "positive" and predicted_label == "positive":
            category = "VP"
        elif true_label == "negative" and predicted_label == "negative":
            category = "VN"
        elif true_label == "positive" and predicted_label == "negative":
            category = "FN"
        elif true_label == "negative" and predicted_label == "positive":
            category = "FP"
        categories[category].append(critique)

        # Analyse mot par mot
        sent = [tok.text for tok in s]
        for pos in tqdm(range(len(s.tokens))):
            word = sent[pos]

            if word not in polarities:
                polarities[word] = 0  
                occurences[word] = 0  

            substracted = flair.data.Sentence(" ".join(sent[:pos] + sent[pos + 1:]))
            flair_sentiment.predict(substracted)
            substracted_score = substracted.labels[0].score

            # Calcul de delta
            delta = sentence_score - substracted_score

            # Changement de polarité
            if s.labels[0].value != substracted.labels[0].value:
                if s.labels[0].value == "POSITIVE":
                    polarities[word] += 1
                else:
                    polarities[word] -= 1

            else:  # Pas de changement de polarité
                if s.labels[0].value == "POSITIVE":
                    if delta < 0:
                        polarities[word] += abs(delta)
                    else:
                        polarities[word] -= abs(delta)
                else:
                    if delta < 0:
                        polarities[word] += abs(delta)
                    else:
                        polarities[word] -= abs(delta)

            occurences[word] += 1  # Compter le nombre d'occurrences du mot

# Moyenne des polarités par mot
for word in polarities.keys():
    polarities[word] /= occurences[word]

# Tri des polarités
sorted_polarities = dict(sorted(polarities.items(), key=lambda item: item[1]))

with open(output_file, "w", encoding="utf-8") as outfile:
    for word, polarity in sorted_polarities.items():
        outfile.write(f"{word} {round(polarity, 3)} {occurences[word]}\n")

for category, critiques in categories.items():
    print(f"{category}: {len(critiques)} critiques")
    print("\n".join(critiques[:3])) 


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
100%|██████████| 278/278 [00:04<00:00, 56.42it/s]

VP: 25 critiques
A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master's of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional 'dream' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell's murals decorating every surface) are terr

In [22]:
print(stop_words)

{"it's", 'ain', 'do', 'until', 'about', 'below', 'each', 'than', "needn't", 'against', "you're", 'themselves', 'she', 'can', 'myself', 'herself', 'haven', "isn't", 're', 'during', 'few', 'will', "hasn't", 'through', 'under', 'both', 'if', 'him', 'needn', 'had', "wouldn't", "she's", 'which', 'whom', "doesn't", "aren't", 'is', 't', 'there', 'on', 'only', "shan't", 'once', 'such', 'her', 'should', 'between', 'ours', 'having', 'was', 'me', "mustn't", 'here', 'all', 'mustn', 'by', 'being', 'then', 'shan', 'why', 'its', 'but', 'aren', 'ourselves', "should've", "don't", 'i', 'don', 'hasn', "you'd", 'further', 'own', 'o', 'from', "you've", 'other', 'at', 'wasn', 'a', "you'll", 'of', 'before', 'after', 'not', "didn't", 'again', 'now', 'did', 'what', 'that', 'shouldn', 'doing', 'how', 'didn', 'nor', 'so', 'couldn', 'doesn', 'out', 'isn', "haven't", 'are', "that'll", 'has', 'yourselves', 'down', 'who', 'those', 'the', 'he', 'to', 'more', 'hadn', "won't", 'no', 'as', 'does', 'for', 'an', 'too', 'o

In [23]:
count_zero_polarity = sum(1 for polarity in polarities.values() if polarity == 0)

print(f"Nombre de mots avec une polarité de 0 : {count_zero_polarity}")

Nombre de mots avec une polarité de 0 : 1800


In [1]:
output_file = "/content/result_out_clean_text.txt"
polarities = {}
with open(output_file, "r", encoding="utf-8") as infile:
    for line in infile:
        word, polarity, occurrences = line.split()
        polarities[word] = float(polarity)

# Trier les polarités (du plus grand au plus petit)
sorted_polarities = sorted(polarities.items(), key=lambda item: item[1], reverse=True)

print("Mot avec la plus grande polarité :")
print(f"{sorted_polarities[0][0]} : {sorted_polarities[0][1]}")  
print("\nMot avec la plus petite polarité :")
print(f"{sorted_polarities[-1][0]} : {sorted_polarities[-1][1]}")  

print("\n20 mots avec la plus grande polarité :")
for word, polarity in sorted_polarities[:20]:
    print(f"{word} : {polarity}")

print("\n20 mots avec la plus petite polarité :")
for word, polarity in sorted_polarities[-20:]:
    print(f"{word} : {polarity}")


Mot avec la plus grande polarité :
likable : 1.0

Mot avec la plus petite polarité :
biggest : -1.0

20 mots avec la plus grande polarité :
likable : 1.0
johanson : 1.0
resurrection : 1.0
tech : 1.0
ole : 1.0
saywould : 1.0
huntif : 1.0
hr : 1.0
liking : 1.0
chiller : 1.0
1940s : 1.0
vanishes : 1.0
ageing : 1.0
discover : 1.0
afterwardsbr : 1.0
hag : 1.0
hunchback : 1.0
spooky : 1.0
belas : 1.0
coffin : 1.0

20 mots avec la plus petite polarité :
timer : -1.0
maggot : -1.0
douses : -1.0
spoilersbr : -1.0
rebecca : -1.0
paranoid : -1.0
jingoistic : -1.0
deceptive : -1.0
operator : -1.0
property : -1.0
coincidence : -1.0
literary : -1.0
maven : -1.0
brief : -1.0
returning : -1.0
neglected : -1.0
kasugi : -1.0
johnrhys : -1.0
badventure : -1.0
biggest : -1.0


In [10]:
noms_a_verifier = [
    "david", "finney", "johanson", "rebecca", "madge", "adam", "emma", "james", "jennifer",
    "christopher", "susan", "john", "mary", "patrick", "george", "lisa", "steve", "robert",
    "katherine", "michael", "paul", "linda", "andrew", "joseph", "nancy", "richard", "sarah",
    "william", "alexander", "benjamin", "elizabeth", "kate", "charles", "victoria", "sophie",
    "natalie", "thomas", "hannah", "will", "alice", "georgia", "sophie", "laura", "lucas"
]


polarites_verifiees = {}

with open("/content/result_out_clean_text.txt", "r", encoding="utf-8") as infile:
    for line in infile:
        word, polarity, occurrences = line.split()
        word = word.lower()  

        if word in noms_a_verifier:
            polarites_verifiees[word] = float(polarity)

# Afficher la polarité des mots vérifiés
for nom in noms_a_verifier:
    if nom in polarites_verifiees:
        print(f"Le mot '{nom}' a une polarité de {polarites_verifiees[nom]}")
    else:
        print(f"Le mot '{nom}' n'a pas été trouvé dans le fichier.")


Le mot 'david' a une polarité de 0.007
Le mot 'finney' a une polarité de 0.031
Le mot 'johanson' a une polarité de 1.0
Le mot 'rebecca' a une polarité de -1.0
Le mot 'madge' a une polarité de 0.022
Le mot 'adam' a une polarité de -0.153
Le mot 'emma' n'a pas été trouvé dans le fichier.
Le mot 'james' a une polarité de 0.018
Le mot 'jennifer' a une polarité de -0.042
Le mot 'christopher' a une polarité de 0.17
Le mot 'susan' a une polarité de 0.294
Le mot 'john' a une polarité de -0.012
Le mot 'mary' a une polarité de 0.0
Le mot 'patrick' n'a pas été trouvé dans le fichier.
Le mot 'george' a une polarité de 0.003
Le mot 'lisa' n'a pas été trouvé dans le fichier.
Le mot 'steve' a une polarité de -0.0
Le mot 'robert' a une polarité de 0.049
Le mot 'katherine' n'a pas été trouvé dans le fichier.
Le mot 'michael' a une polarité de 0.0
Le mot 'paul' a une polarité de -0.0
Le mot 'linda' n'a pas été trouvé dans le fichier.
Le mot 'andrew' a une polarité de 0.032
Le mot 'joseph' n'a pas été tr

In [23]:
words_to_compare = [
    "happy", "sad", "angry", "love", "hate", "excited", "bored", "fear",
    "surprised", "nervous",  "confused", "afraid",
    "proud",  "trust", "disappointed","desperate", "unhappy", "hopeless", "gloomy","sympathetic", "bitter"
]
polarites_verifiees = {}

with open("/content/result_out_clean_text.txt", "r", encoding="utf-8") as infile:
    for line in infile:
        word, polarity, occurrences = line.split()
        word = word.lower() 

        # Si le mot est dans la liste des mots à comparer, ajouter sa polarité
        if word in words_to_compare:
            polarites_verifiees[word] = float(polarity)

for word in words_to_compare:
    if word in polarites_verifiees:
        print(f"Le mot '{word}' a une polarité de {polarites_verifiees[word]}")
    else:
        print(f"Le mot '{word}' n'a pas été trouvé dans le fichier.")

Le mot 'happy' a une polarité de 0.001
Le mot 'sad' a une polarité de -0.0
Le mot 'angry' a une polarité de -0.0
Le mot 'love' a une polarité de 0.04
Le mot 'hate' a une polarité de 0.011
Le mot 'excited' a une polarité de 0.003
Le mot 'bored' a une polarité de -0.0
Le mot 'fear' a une polarité de 0.001
Le mot 'surprised' a une polarité de -0.083
Le mot 'nervous' a une polarité de -0.108
Le mot 'confused' a une polarité de 0.0
Le mot 'afraid' a une polarité de -0.499
Le mot 'proud' a une polarité de 0.008
Le mot 'trust' a une polarité de -0.005
Le mot 'disappointed' a une polarité de 0.024
Le mot 'desperate' a une polarité de -0.011
Le mot 'unhappy' a une polarité de 0.013
Le mot 'hopeless' a une polarité de 0.0
Le mot 'gloomy' a une polarité de -0.02
Le mot 'sympathetic' a une polarité de -0.002
Le mot 'bitter' a une polarité de -0.001


In [32]:
religious_words = [
    "christian", "jesus", "church", "muslim",
    "jew","temple", "faith", "ritual",
    "jesus","moses", "mary", "peace", "forgiveness", "grace", "soul"
]

polarites_verifiees = {}

with open("/content/result_out_clean_text.txt", "r", encoding="utf-8") as infile:
    for line in infile:
        word, polarity, occurrences = line.split()
        word = word.lower()  

        # Si le mot est dans la liste des mots religieux, ajouter sa polarité
        if word in religious_words:
            polarites_verifiees[word] = float(polarity)

for word in religious_words:
    if word in polarites_verifiees:
        print(f"Le mot '{word}' a une polarité de {polarites_verifiees[word]}")
    else:
        print(f"Le mot '{word}' n'a pas été trouvé dans le fichier.")


Le mot 'christian' a une polarité de -0.007
Le mot 'jesus' a une polarité de -0.0
Le mot 'church' a une polarité de 0.005
Le mot 'muslim' a une polarité de -0.007
Le mot 'jew' a une polarité de -0.0
Le mot 'temple' a une polarité de 0.01
Le mot 'faith' a une polarité de 0.14
Le mot 'ritual' a une polarité de -0.018
Le mot 'jesus' a une polarité de -0.0
Le mot 'moses' a une polarité de 0.0
Le mot 'mary' a une polarité de 0.0
Le mot 'peace' a une polarité de 0.045
Le mot 'forgiveness' a une polarité de 0.0
Le mot 'grace' a une polarité de 0.0
Le mot 'soul' a une polarité de -0.0


In [5]:
mental_health_words = [
"depression",'stress',"addiction","loneliness","guilt","anger","fear","frustration","treatment","suicide"
]


polarites_verifiees = {}

with open("/content/result_out_clean_text.txt", "r", encoding="utf-8") as infile:
    for line in infile:
        word, polarity, occurrences = line.split()
        word = word.lower()  
        # Si le mot est dans la liste des mots associés au racisme, ajouter sa polarité
        if word in mental_health_words:
            polarites_verifiees[word] = float(polarity)

for word in mental_health_words:
    if word in polarites_verifiees:
        print(f"Le mot '{word}' a une polarité de {polarites_verifiees[word]}")
    else:
        print(f"Le mot '{word}' n'a pas été trouvé dans le fichier.")


Le mot 'depression' a une polarité de -0.007
Le mot 'stress' a une polarité de -0.01
Le mot 'addiction' a une polarité de -0.013
Le mot 'loneliness' a une polarité de -0.027
Le mot 'guilt' a une polarité de 0.003
Le mot 'anger' a une polarité de 0.0
Le mot 'fear' a une polarité de 0.001
Le mot 'frustration' a une polarité de 0.0
Le mot 'treatment' a une polarité de -0.003
Le mot 'suicide' a une polarité de -0.0
